# Chatbot

Reference: python.langchain.com/v0.2/docs/tutorials/chatbot

pip install langchain langchain-core langchain-community
pip install -qU langchain-openai

## Read Secrets

In [70]:
import configparser
import os


config = configparser.ConfigParser()
config.read('./creds.cfg')
os.environ["OPENAI_API_KEY"]=config["OPENAI"]["OPENAI_API_KEY"]
os.environ["OPENAI_API_ENDPOINT"]=config["OPENAI"]["OPENAI_API_ENDPOINT"]

## Setup Model

In [71]:
from langchain_openai import ChatOpenAI


model = ChatOpenAI(model="gpt-3.5-turbo")

In [72]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I am Jai")])

AIMessage(content="Hello Jai! It's nice to meet you. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 13, 'total_tokens': 31}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-48b3dfa3-e345-4fc1-b628-ecd884a43643-0', usage_metadata={'input_tokens': 13, 'output_tokens': 18, 'total_tokens': 31})

## Add Message History

In [73]:
from langchain_core.chat_history import (
    BaseChatMessageHistory, 
    InMemoryChatMessageHistory
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

## Prompt Templates

In [75]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import SystemMessage, trim_messages, AIMessage

trimmer = trim_messages(
    max_tokens = 65,
    strategy = "last",
    token_counter = model, 
    include_system = True, 
    allow_partial=False, 
    start_on="human"
    )
   
messages = []

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all question to the best of your ability in {language}."
        ),
        MessagesPlaceholder(variable_name="messages")
        
    ]
)

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt | model
)

In [76]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")
config = {"configurable": {"session_id":"abc21"}}

In [77]:
response = with_message_history.invoke({"messages": messages + [HumanMessage(content="Hi I am Jai")], "language":"English"}, config=config)
response.content

'¡Hola Jai! ¿En qué puedo ayudarte hoy?'

In [78]:
response = with_message_history.invoke({"messages":messages + [HumanMessage(content="What is my name")], "language":"English"}, config=config)
response.content

'Your name is Jai.'

In [79]:
response = with_message_history.invoke({"messages":messages + [HumanMessage(content="My favorite color is blue")], "language":"English"}, config=config)
response.content

"That's great! Blue is a popular favorite color for many people."

In [80]:
response = with_message_history.invoke({"messages":messages + [HumanMessage(content="I enjoy fishing")], "language":"English"}, config=config)
response.content

'Fishing is a relaxing and enjoyable hobby for many people. It can be a great way to spend time outdoors and unwind.'

In [81]:
response = with_message_history.invoke({"messages":messages + [HumanMessage(content="What's my name?")], "language":"English"}, config=config)
response.content

"I'm sorry, but I don't have access to that information. If you'd like, you can share your name with me."